In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("broadcast_join").getOrCreate()

orders_rdd = spark.sparkContext.textFile("hdfs://localhost:9000/user/tkm/data/sample_superstore/orders.csv")
returns_rdd =spark.sparkContext.textFile("hdfs://localhost:9000/user/tkm/data/sample_superstore/returns.csv")

header_orders =  orders_rdd.first()
header_returns =  returns_rdd.first() #these two are action but create a single default stage and 1 task  


#narrow transformation
orders_filtered_rdd = orders_rdd.filter(lambda x : (x!=header_orders))
returns_filtered_rdd = returns_rdd.filter(lambda x : (x!=header_returns)) #narrow transformation

#narrow transformation
mapped_orders_filtered_rdd = orders_filtered_rdd.map(lambda x : ((x.split(",")[1]),(x.split(",")[15])))
mapped_returns_filtered_rddd = returns_filtered_rdd.map(lambda x :((x.split(",")[1]),(x.split(",")[0])))


In [2]:
mapped_return_broadcast = spark.sparkContext.broadcast(mapped_returns_filtered_rddd.collect())

In [ ]:
mapped_return_broadcast.take()

In [3]:
def status(order_id):
    try:
        return mapped_return_broadcast.value[order_id]
    except:
        return -1

In [ ]:
#wide transformation
joined_rdd = mapped_orders_filtered_rdd.map(lambda x : (x[0], x[1], ( status( int(x[0]) ) )    )    )

#action
joined_rdd.saveAsTextFile("C:\\Users\\TARUN\\Desktop\\Pyspark\\superstore\\broadcast_join_results")